<a href="https://colab.research.google.com/github/nurimammasri/Dicoding-Belajar-Pengembangan-Machine-Learning/blob/main/09_NLP_multclassTextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
import pandas as pd
# imdb_indonesian_movies_2.csv
# df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = pd.read_csv('https://raw.githubusercontent.com/nurimammasri/Dicoding-Belajar-Pengembangan-Machine-Learning/main/data/imdb_indonesian_movies_2.csv')
df = df.drop(columns = ['judul_film'])

In [3]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Melakukan proses one-hot-encoding karena dataset nya berupa kategorikal.

In [4]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis = 1)
df_baru = df_baru.drop(columns = 'genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [5]:
sinposis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Pembagian dataset menjadi Train dan Test

In [6]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinposis, label, test_size = 0.2)

Tokenisasi

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 5000, oov_token = 'x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. 

In [8]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 5000, output_dim = 16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'softmax')
])
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [9]:
num_epochs = 30
history = model.fit(
    padded_latih,
    label_latih,
    epochs = num_epochs,
    validation_data = (padded_test, label_test),
    verbose = 2
)

Epoch 1/30
26/26 - 31s - loss: 1.6115 - accuracy: 0.2040 - val_loss: 1.6151 - val_accuracy: 0.1741
Epoch 2/30
26/26 - 28s - loss: 1.6024 - accuracy: 0.2363 - val_loss: 1.6186 - val_accuracy: 0.1642
Epoch 3/30
26/26 - 28s - loss: 1.5224 - accuracy: 0.3358 - val_loss: 1.6379 - val_accuracy: 0.1692
Epoch 4/30
26/26 - 28s - loss: 1.2146 - accuracy: 0.4042 - val_loss: 1.8831 - val_accuracy: 0.2090
Epoch 5/30
26/26 - 28s - loss: 0.9398 - accuracy: 0.5323 - val_loss: 2.3432 - val_accuracy: 0.2438
Epoch 6/30
26/26 - 28s - loss: 0.7657 - accuracy: 0.5771 - val_loss: 2.7678 - val_accuracy: 0.2786
Epoch 7/30
26/26 - 28s - loss: 0.6256 - accuracy: 0.6928 - val_loss: 3.2899 - val_accuracy: 0.2488
Epoch 8/30
26/26 - 29s - loss: 0.5998 - accuracy: 0.7488 - val_loss: 2.8918 - val_accuracy: 0.2587
Epoch 9/30
26/26 - 28s - loss: 0.4642 - accuracy: 0.8607 - val_loss: 3.4742 - val_accuracy: 0.2736
Epoch 10/30
26/26 - 29s - loss: 0.2620 - accuracy: 0.9216 - val_loss: 4.5621 - val_accuracy: 0.2736
Epoch 11/